# Fast simulation perfromance parameterization 
Expert  configurable event display for fast simulzation studies  


## Contents 
[Import modules](#import) <br>
[Load trees, set aliases](#load) <br>
[XYZ display pt](#XYZdisplayPt) <br>
[XYZ display pt](#XYZdisplaytrack) <br>

In [ ]:
from bokeh.io import output_notebook
from RootInteractive.Tools.aliTreePlayer import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc
import numpy as np
import os
import sys
from ROOT import TFile, gSystem
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter, RandomForest
import plotly.express as px
import time
import pickle
from IPython.display import Image, clear_output
from RootInteractive.InteractiveDrawing.bokeh.CDSCompress import *
#
from fastSimulation import *

In [ ]:
from RootInteractive.MLpipeline.MIForestErrPDF import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

## 1.) Load trees and define derived variables:
<a id='load'></a>

* Covaraince matrix of Alice kalman track - stored as an array 
* ```(y,z,sing*phi),tgl(theta),qPt) -> (0,2,5,9,14)```
 * 0   
 * 1 2
 * 3 4 5 
 * 6 7 8 9
 * 10 11 12 13 14 

In [ ]:
#inputData = "/home2/miranov/github/fastMCKalman/data/alice3/fastParticle.list";
#inputData = "${fastMCKalman}/data/alice3/fastParticle.list";
inputData = "/home2/miranov/github/fastMCKalman/data/testTPCSA/fastParticle.list"
#inputData = "/home2/miranov/github/fastMCKalman/data/run2/fastParticle.list";

In [ ]:
ROOT.initTreeFast(inputData)
tree=ROOT.treeFast

In [ ]:
tree.SetMarkerStyle(25)
tree.SetMarkerSize(0.5)
ROOT.setAliases(tree)
tree.SetAlias("snp0","part.fParamIn[0].fP[2]")
tree.SetAlias("qPt","part.fParamMC[0].fP[4]")
tree.SetAlias("ptMC","part.fParamMC[0].fData.Pt()")
tree.SetAlias("ptIn","part.fParamIn[1].fData.Pt()")
tree.SetAlias("ptMCP","part.fParamMC[].fData.Pt()")
tree.SetAlias("tglMC","part.fParamMC[0].fP[3]")
tree.SetAlias("tglIn","part.fParamIn[1].fP[3]")
#
tree.SetAlias("sigmaY0","sqrt(part.fParamIn[1].fC[0])")
tree.SetAlias("sigmaZ0","sqrt(part.fParamIn[1].fC[2])")
tree.SetAlias("sigmaqPt0","sqrt(part.fParamIn[1].fC[14])")
#
tree.SetAlias("sigmaY0Rot","sqrt(part.fParamInRot[1].fC[0])")
tree.SetAlias("sigmaZ0Rot","sqrt(part.fParamInRot[1].fC[2])")
tree.SetAlias("sigmaqPt0Rot","sqrt(part.fParamInRot[1].fC[14])")
#
tree.SetAlias("gzMC","(part.fParamMC[].fP[1])")
tree.SetAlias("gzIn","(part.fParamIn[].fP[1])")
tree.SetAlias("rMC","sqrt(gxMC**2+gyMC**2)")
tree.SetAlias("snpMC","(part.fParamMC[].fP[2])")
tree.SetAlias("pIndex","Iteration$")
tree.SetAlias("trackID","i")
tree.SetAlias("snpMax","abs(part.fParamMC[fMaxLayer].fP[2])")
#
tree.SetAlias("meanptMC","part.getMean(0+0,0)");
tree.SetAlias("meanptMCInv","part.getMean(1+0,0)");
tree.SetAlias("meandEdxExp","part.getMean(2+0,0)");
tree.SetAlias("meandEdxExpInv","part.getMean(3+0,0)");
tree.SetAlias("meanStat4","part.getMean(4+0,0)");
tree.SetAlias("X0Norm","geom.fLayerX0.fData[0]/7.8350968e-05");
tree.SetAlias("sigmaRPhi","geom.fLayerResolRPhi.fData[0]");
tree.SetAlias("sigmaPtRel0","sigmaqPt0*ptMC");
tree.SetAlias("sigmaPtRel0Mean","sigmaqPt0/meanptMCInv");
#
tree.SetAlias("fitCut","ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5")
#tree.SetAlias("Larm","fLengthIn")    # the number of clusters not corresponding to length in the reco
tree.SetAlias("Larm","getStat(2)") 
c=ROOT.TCanvas()


In [ ]:
tree.SetAlias("dir1","part.fParamIn[1].fData.GetDirectionSign()")
#tree.Draw("part.fParamIn[1].fData.GetDirectionSign()")

## Load panda from tree

In [ ]:
%%time
tree.SetAlias("fitCut","ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5&&fMaxLayer>2")
#tree.SetAlias("meanStat0","getStat(0)")
defaultCutTrack="Larm>5&&fMaxLayer>1&&sigmaqPt0>0&&sigmaPtRel0>0"

kineParam=["sigmaPtRel0","qPt","ptMC",
                           "sigmaY0","sigmaZ0","sigmaqPt0",
                           "sigmaY0Rot","sigmaZ0Rot","sigmaqPt0Rot",
                           "pidCode","isSecondary","fMaxLayer","dEdxExp",
                          "snpMax","fMassMC","dir1","snp0","meanStat4"]
resolParam=["meanptMCInv","meandEdxExpInv","X0Norm","tglMC","Larm","sigmaRPhi"]

dfRes=tree2Panda(tree,kineParam+resolParam,
                 defaultCutTrack,columnMask=[["_fElements",""]],nEntries=10000)
dfFit=dfRes[0:dfRes.shape[0]//2]
dfTest=dfRes[dfRes.shape[0]//2:]
print(dfRes.shape)
#dfResFit=dfRes[:dfRes.shape[1]//2]
#dfResTest=dfRes[dfRes.shape[1]//2:]

## Make high pt scaling

In [ ]:
from sklearn.linear_model import LinearRegression
#dfResMLDraw["FitRatioRF"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")
dfRes["qptResolHPT"]=dfRes.eval("(sigmaRPhi+0.001)/Larm**2.5")
dfResHPT=dfRes.query("ptMC>4&Larm>10&isSecondary==0")
linearHPT=LinearRegression()
linearHPT.fit(dfResHPT[["qptResolHPT"]],dfResHPT["sigmaqPt0"])
dfRes["qptResolHPTFit"]=linearHPT.predict(dfRes[["qptResolHPT"]])
dfResHPT["qptResolHPTFit"]=linearHPT.predict(dfResHPT[["qptResolHPT"]])
dfResHPT.plot.scatter(y=["sigmaqPt0"],x=["qptResolHPTFit"],figsize=(8, 5),grid=True)

# Make low pt scaling

In [ ]:
from sklearn.linear_model import LinearRegression
#dfResMLDraw["FitRatioRF"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")
dfRes["pMC"]=dfRes.eval("sqrt(1+tglMC**2)/meanptMCInv")
dfRes["Beta"]=dfRes.eval("sqrt(pMC**2/(pMC**2+fMassMC**2))")
dfRes["dEdxExpInv"]=1/dfRes["dEdxExp"]
dfRes["offset"]=1
#
dfRes["sigmaPtRel0LPTV0"]=dfRes.eval("sqrt(X0Norm*dEdxExp)/sqrt(Larm)")              # formula using the parater at the beginning
dfRes["sigmaPtRel0LPTV1"]=dfRes.eval("(ptMC*meanptMCInv)*sqrt((X0Norm/meandEdxExpInv))/sqrt(Larm)")    # formula using the Mean dEdx and pt  along trajectory
dfRes["sigmaPtRel0LPTV2"]=dfRes.eval("pMC*sqrt(X0Norm*meanStat4)/sqrt(Larm)")      # mean formula

dfRes["sigmaPtRel0LPTV1R"]=dfRes["sigmaPtRel0LPTV1"]/dfRes["sigmaPtRel0LPTV0"]
dfRes["sigmaPtRel0LPTV2R"]=dfRes["sigmaPtRel0LPTV2"]/dfRes["sigmaPtRel0LPTV0"]
dfRes["sigmaPtRel0LPTV2R1"]=dfRes["sigmaPtRel0LPTV2"]/dfRes["sigmaPtRel0LPTV1"]
dfResLPT=dfRes.query("ptMC<1&Larm>30&isSecondary==0&sigmaPtRel0<1&abs(tglMC)<1.5")
#
linearLPTV0=LinearRegression()
linearLPTV1=LinearRegression()
linearLPTV2=LinearRegression()
linearLPTV0.fit(dfResLPT[["sigmaPtRel0LPTV0","offset"]],dfResLPT["sigmaPtRel0"])
linearLPTV1.fit(dfResLPT[["sigmaPtRel0LPTV1","offset"]],dfResLPT["sigmaPtRel0"])
linearLPTV2.fit(dfResLPT[["sigmaPtRel0LPTV2","offset"]],dfResLPT["sigmaPtRel0"])
#
dfRes["sigmaPtRel0LPTV0Fit"]=linearLPTV0.predict(dfRes[["sigmaPtRel0LPTV0","offset"]])
dfRes["sigmaPtRel0LPTV1Fit"]=linearLPTV1.predict(dfRes[["sigmaPtRel0LPTV1","offset"]])
dfRes["sigmaPtRel0LPTV2Fit"]=linearLPTV2.predict(dfRes[["sigmaPtRel0LPTV2","offset"]])
dfResLPT["sigmaPtRel0LPTV0Fit"]=linearLPTV0.predict(dfResLPT[["sigmaPtRel0LPTV0","offset"]])
dfResLPT["sigmaPtRel0LPTV1Fit"]=linearLPTV1.predict(dfResLPT[["sigmaPtRel0LPTV1","offset"]])
dfResLPT["sigmaPtRel0LPTV2Fit"]=linearLPTV2.predict(dfResLPT[["sigmaPtRel0LPTV2","offset"]])
#
dfResLPT.plot.scatter(x=["sigmaPtRel0LPTV0Fit"],y=["sigmaPtRel0"],figsize=(8, 5),grid=True)
dfResLPT.plot.scatter(x=["sigmaPtRel0LPTV1Fit"],y=["sigmaPtRel0"],figsize=(8, 5),grid=True)
dfResLPT.plot.scatter(x=["sigmaPtRel0LPTV2Fit"],y=["sigmaPtRel0"],figsize=(8, 5),grid=True)
dfResLPT.plot.scatter(x=["sigmaPtRel0LPTV0"],y=["sigmaPtRel0LPTV1R"],figsize=(8, 5),grid=True)
dfResLPT.plot.scatter(x=["sigmaPtRel0LPTV0"],y=["sigmaPtRel0LPTV2R"],figsize=(8, 5),grid=True)
dfResLPT.plot.scatter(x=["sigmaPtRel0LPTV0"],y=["sigmaPtRel0LPTV2R1"],figsize=(8, 5),grid=True)
#dfRes.plot.scatter(x=["dEdxExpInv"],y=["meandEdxExpInv"],figsize=(8, 5),grid=True)
#dfRes.plot.scatter(x=["qPt"],y=["meanptMCInv"],figsize=(8, 5),grid=True)

In [ ]:
dfRes["offset"]

In [ ]:
linearLPTV1.coef_
linearLPTV1.intercept_

In [ ]:
dfRes["sigmaPtRel0LPTV0LogRatio"]=np.log(dfRes["sigmaPtRel0LPTV0Fit"]/dfRes["sigmaPtRel0"])
dfRes["sigmaPtRel0LPTV1LogRatio"]=np.log(dfRes["sigmaPtRel0LPTV1Fit"]/dfRes["sigmaPtRel0"])
dfRes["sigmaPtRel0LPTV2LogRatio"]=np.log(dfRes["sigmaPtRel0LPTV2Fit"]/dfRes["sigmaPtRel0"])
dfRes["sigmaPtRel0LPTV2LogRatio1"]=np.log((dfRes["sigmaPtRel0LPTV2Fit"]-0.01)/dfRes["sigmaPtRel0"])
dfRes.query("ptMC<1&Larm>10&isSecondary==0&sigmaPtRel0<1&abs(tglMC)<1.5&sigmaPtRel0<0.1")[["sigmaPtRel0LPTV0LogRatio","sigmaPtRel0LPTV1LogRatio","sigmaPtRel0LPTV2LogRatio","sigmaPtRel0LPTV2LogRatio1"]].hist(range=(-2,2),bins=100)

dfRes.query("ptMC<1&Larm>10&isSecondary==0&sigmaPtRel0<1&abs(tglMC)<1.5&sigmaPtRel0<0.1")[["sigmaPtRel0LPTV0LogRatio","sigmaPtRel0LPTV1LogRatio","sigmaPtRel0LPTV2LogRatio"]].std()
dfRes.query("ptMC<1&Larm>10&isSecondary==0&sigmaPtRel0<1&abs(tglMC)<1.5&sigmaPtRel0<0.1").plot.scatter(x=["sigmaPtRel0"],y=["sigmaPtRel0LPTV2LogRatio"],figsize=(8, 5),grid=True)
dfRes.query("ptMC<1&Larm>10&isSecondary==0&sigmaPtRel0<1&abs(tglMC)<1.5&sigmaPtRel0<0.1").plot.scatter(x=["sigmaPtRel0"],y=["sigmaPtRel0LPTV2LogRatio1"],figsize=(8, 5),grid=True)

dfRes.query("ptMC<1&Larm>10&isSecondary==0&sigmaPtRel0<1&abs(tglMC)<1.5&sigmaPtRel0<0.1")[["sigmaPtRel0LPTV0LogRatio","sigmaPtRel0LPTV1LogRatio","sigmaPtRel0LPTV2LogRatio","sigmaPtRel0LPTV2LogRatio1"]].std()

In [ ]:
linearLPTMean.coef_

In [ ]:
%%time
n_estimators=200
n_jobs=4
max_depth=10
#
dfRes['sigmaPtRelS']=dfRes["sigmaPtRel0"]*(dfRes["Larm"]**1.5)
dfRes['sigmaPtRelSL']=np.log(dfRes["sigmaPtRelS"])

resolParam=["meanptMCInv","meandEdxExpInv","X0Norm","tglMC","Larm","sigmaRPhi","snp0"]
dfResML=dfRes
npoints=dfRes.shape[0]
varFit='sigmaPtRelS'
variableX=resolParam
#
regressorRF0 = RandomForestRegressor(n_estimators =n_estimators,n_jobs=n_jobs,max_depth=max_depth,max_samples=0.5)
regressorRF1 = RandomForestRegressor(n_estimators =n_estimators,n_jobs=n_jobs,max_depth=None,max_samples=0.5)
regressorRFL = RandomForestRegressor(n_estimators =n_estimators,n_jobs=n_jobs,max_depth=None,max_samples=0.5)
regressorRF0.fit(dfRes[resolParam],dfRes[varFit])
regressorRF1.fit(dfRes[resolParam],dfRes[varFit])
regressorRFL.fit(dfRes[resolParam],dfRes["sigmaPtRelSL"])
#regressorRF1.fit(dfTrain0[variables],dfTrain0["LinearDeltaBase"])
#regressorRF1.importance_

In [ ]:
# The mean estimater has snaller simga than median estimator

In [ ]:
mean_depth=10
niter_max=10
miErrPDF=MIForestErrPDF("Regressor",{"mean_depth":mean_depth, "n_jobs":n_jobs,"niter_max":niter_max })
miErrPDF.fit(dfRes[resolParam],dfRes["sigmaPtRelSL"])
#
outputErr={}
x=[0.3,0.5,0.7]
#miErrPDF.predictReducibleError(dfResFit[resolParam],  outputErr, 32, x)
statDictionary={"mean":0,"median":0, "std":0}
statOut=predictRFStat(regressorRF0,dfRes[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
#

In [ ]:
statDictionary={"mean":0,"median":0, "std":0}
miErrPDF.predictStat(dfRes[resolParam],statDictionary)
dfRes["sigmaPtRelE_Mean"]=np.exp(statDictionary["mean"])/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRelE_Median"]=np.exp(statDictionary["median"])/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRelE_std"]=np.exp(statDictionary["std"])/(dfRes["Larm"]**1.5)

In [ ]:
regressorRF1.feature_importances_

In [ ]:
%%time
statDictionary={"mean":0, "std":0, "median":0, "quantile":[0.1,0.5,0.9]}
predictRF0=predictRFStat(regressorRF0,dfRes[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
dfRes["sigmaPtRel0_Mean"]=statDictionary["mean"]/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRel0_Mean0"]=regressorRF0.predict(dfRes[resolParam])/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRel0_Median"]=statDictionary["median"]/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRel0_std"]=statDictionary["std"]/(dfRes["Larm"]**1.5)
#
predictRF1=predictRFStat(regressorRF1,dfRes[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
dfRes["sigmaPtRel1_Mean1"]=regressorRF1.predict(dfRes[resolParam])/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRel1_Mean"]=statDictionary["mean"]/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRel1_Median"]=statDictionary["median"]/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRel1_std"]=statDictionary["std"]/(dfRes["Larm"]**1.5)
#
predictRFL=predictRFStat(regressorRFL,dfRes[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
dfRes["sigmaPtRelL_Mean1"]=np.exp(regressorRF1.predict(dfRes[resolParam]))/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRelL_Mean"]=np.exp(statDictionary["mean"])/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRelL_Median"]=np.exp(statDictionary["median"])/(dfRes["Larm"]**1.5)
dfRes["sigmaPtRelL_std"]=np.exp(statDictionary["std"])/(dfRes["Larm"]**1.5)
#
print("sigmaPtRel0_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel0_Mean"])).std())
print("sigmaPtRel1_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel1_Mean"])).std())
print("sigmaPtRelL_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelL_Mean"])).std())
print("sigmaPtRelE_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelE_Mean"])).std())

print("sigmaPtRel0_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel0_Median"])).std())
print("sigmaPtRel1_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel1_Median"])).std())
print("sigmaPtRelL_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelL_Median"])).std())
print("sigmaPtRelE_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelE_Median"])).std())



In [ ]:
dfTest

In [ ]:
%%time
statDictionary={"mean":0, "std":0, "median":0, "quantile":[0.1,0.5,0.9]}
predictRF0=predictRFStat(regressorRF0,dfTest[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
dfTest["sigmaPtRel0_Mean"]=statDictionary["mean"]/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRel0_Mean0"]=regressorRF0.predict(dfTest[resolParam])/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRel0_Median"]=statDictionary["median"]/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRel0_std"]=statDictionary["std"]/(dfTest["Larm"]**1.5)
#
predictRF1=predictRFStat(regressorRF1,dfTest[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
dfTest["sigmaPtRel1_Mean1"]=regressorRF1.predict(dfTest[resolParam])/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRel1_Mean"]=statDictionary["mean"]/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRel1_Median"]=statDictionary["median"]/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRel1_std"]=statDictionary["std"]/(dfTest["Larm"]**1.5)
#
predictRFL=predictRFStat(regressorRFL,dfTest[resolParam].to_numpy(dtype=np.float32),statDictionary,n_jobs)
dfTest["sigmaPtRelL_Mean1"]=np.exp(regressorRF1.predict(dfTest[resolParam]))/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRelL_Mean"]=np.exp(statDictionary["mean"])/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRelL_Median"]=np.exp(statDictionary["median"])/(dfTest["Larm"]**1.5)
dfTest["sigmaPtRelL_std"]=np.exp(statDictionary["std"])/(dfTest["Larm"]**1.5)
#
print("sigmaPtRel0_Mean",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRel0_Mean"])).std())
print("sigmaPtRel1_Mean",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRel1_Mean"])).std())
print("sigmaPtRelL_Mean",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRelL_Mean"])).std())
#print("sigmaPtRelE_Mean",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRelE_Mean"])).std())

print("sigmaPtRel0_Median",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRel0_Median"])).std())
print("sigmaPtRel1_Median",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRel1_Median"])).std())
print("sigmaPtRelL_Median",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRelL_Median"])).std())
#print("sigmaPtRelE_Median",(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRelE_Median"])).std())

In [ ]:
# print mean resol
print("sigmaPtRel0_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel0_Mean"])).std())
print("sigmaPtRel1_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel1_Mean"])).std())
print("sigmaPtRelL_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelL_Mean"])).std())
print("sigmaPtRelE_Mean",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelE_Mean"])).std())

print("sigmaPtRel0_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel0_Median"])).std())
print("sigmaPtRel1_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRel1_Median"])).std())
print("sigmaPtRelL_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelL_Median"])).std())
print("sigmaPtRelE_Median",(np.log(dfRes["sigmaPtRel0"]/dfRes["sigmaPtRelE_Median"])).std())

In [ ]:
import scipy.stats as stats
stats.mstats.trimmed_std(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRel1_Median"]),limits=(0.05,0.05))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#rfVars=['sigmaPtRel1_Mean','sigmaPtRel1_Median']
#dfRes["sigmaPtRel1_Median"]
#dfRes["sigmaPtRel0"].hist(range=[-0.,0.5],bins=100)
(np.log(dfTest["sigmaPtRel0"]/dfTest["sigmaPtRel1_Median"])).hist(range=[-0.5,0.5],bins=100)

#dfRes.query("sigmaPtRel1_std<0.3&sigmaPtRelL_Median<0.1").plot.scatter(x=["sigmaPtRelL_Median"],y=["sigmaPtRel0"],figsize=(30, 10),grid=True)
dfRes.query("sigmaPtRel1_std<0.3&sigmaPtRel1_Median<0.2").plot.scatter(x=["sigmaPtRel1_Mean"],y=["sigmaPtRelL_Mean"],figsize=(30, 10),grid=True)
#dfRes.plot(x="tglMC",y="tglMC",figsize=(30, 6),grid=True)
#dfResML.plot(x="sigmaPtRel1_Median", y="sigmaPtRel1_Median")

In [ ]:
dfResMLDraw=dfRes
dfResMLDraw["FitRatioRF0"]=dfResMLDraw.eval("log(sigmaPtRel0/sigmaPtRel0_Median)")
dfResMLDraw["FitRatioRF1"]=dfResMLDraw.eval("log(sigmaPtRel0/sigmaPtRel1_Median)")
dfResMLDraw["FitRatioRF"]=dfResMLDraw.eval("log(sigmaPtRel0/sigmaPtRel1_Median)")
dfResMLDraw["FitRatioRFC"]=abs()

## Resolution plot - outliers in covariance matrix could be too small
* resolution at layer 0 as obtained by covariance matrix 
* problem similar in 2 rotation schema
  * rotate to local after each propagation
  * only one rotation
  
###resolParam=["meanptMCInv","meandEdxExpInv","X0Norm","tglMC","Larm","sigmaRPhi"]  

In [ ]:
arrayCompressionRelative10=[(".*",[("relative",10), ("code",0), ("zip",0), ("base64",0)])]
arrayCompressionRelative8=[(".*",[("relative",8), ("code",0), ("zip",0), ("base64",0)])]
output_file("fastSimulDisplayResolution.html")
dfResMLDraw=dfResML
# derived variables

#dfResMLDraw["FitRatioRF"]=dfResMLDraw.eval("log(sigmaPtRel0/RF)")
dfResMLDraw["pMC"]=dfResMLDraw.eval("sqrt(1+tglMC**2)*ptMC")
dfResMLDraw["Beta"]=dfResMLDraw.eval("sqrt(pMC**2/(pMC**2+fMassMC**2))")
# qpt scaling
dfResMLDraw["qptResolHPT"]=dfResMLDraw.eval("(sigmaRPhi+0.0001)*sqrt(720./(Larm+4))/(0.3*0.5*(Larm)**(2.))")
dfResMLDraw["qptResolLPT"]=dfResMLDraw.eval("X0Norm*(0.2*0.5/Beta)*sqrt(Larm)")
# pt scaling
dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*0.01*(sigmaRPhi+0.0001)*sqrt(720./(Larm+4))/(0.3*0.5*(Larm/100.)**(2.)))")
dfResMLDraw["ptResolHPT0"]=dfResMLDraw.eval("(sigmaRPhi+0.0001)/((Larm/100.)**2.5)")
dfResMLDraw["ptResolLPT"]=dfResMLDraw.eval("(0.2*0.5/Beta)*sqrt(Larm/100.)")
dfResMLDraw["FitRatioHPT"]=dfResMLDraw.eval("log(0+sigmaPtRel0/ptResolHPT)")
dfResMLDraw["FitRatioLPT"]=dfResMLDraw.eval("log(0+sigmaPtRel0/ptResolLPT)")

# define categorical
dfResMLDraw["isHighPt"]=dfResMLDraw.eval("ptMC>2")
dfResMLDraw["X0NormC"]=dfResMLDraw["X0Norm"].astype("int").astype("category")
dfResMLDraw["sigmaRPhiC"]=dfResMLDraw.eval("sigmaRPhi/0.025").astype("int").astype("category")
#remove infinities
dfResMLDraw.loc[ abs(dfResMLDraw.FitRatioHPT)==np.inf]=0
# define residulas between resolutio and expectation
#dfResMLDraw["sigmaPtRel0"]=dfResMLDraw.eval("sigmaPtRel0/sigmaPtRel0RF")

In [ ]:
# some command to check infinities
#dfResMLDraw['FitRatioHPT'] = np.replace(dfResMLDraw['FitRatioHPT'],np.inf, 0)
#dfResMLDraw.loc[ abs(dfResMLDraw.FitRatioHPT)==np.inf]
#(abs(dfResMLDraw['FitRatioHPT'])==np.inf).sum()


In [ ]:
parameterArray = [
    {"name": "colorZPt", "value":"Larm", "options":["dEdxExp","X0Norm","pidCode","Larm","ptMC","ptResolHPT","ptResolLPT","sigmaRPhi","ptResolHPT0","isSecondary"]},
    {"name": "colorZqPt", "value":"Larm", "options":["dEdxExp","X0Norm","pidCode","Larm","ptMC","ptResolHPT","ptResolLPT","sigmaRPhi"]},
    {"name": "markerSize", "value":7, "range":[0, 30]},
    {"name": "legendFontSize", "value":"13px", "options":["9px", "11px", "13px", "15px", "17px", "19px"]},  
]

widgetParams=[
    #
    ['select',["colorZPt"], {"callback": "parameter", "default": 2}],
    ['slider',["markerSize"], {"callback": "parameter"}],
    ['select',["legendFontSize"], {"callback": "parameter", "default": 2}],
    #infironment
    ['range', ['sqrt(dEdxExp)']],
    ['range', ['meandEdxExpInv']],
    ['range', ['X0Norm']],
    ['range', ['pidCode']],
    ['range', ['sigmaRPhi']],
    #
    ['range', ['ptMC']],
    ['range', ['qPt']],
    ['range', ['tglMC']],
    ['range', ['Larm']],
    #
    ["range", ['FitRatioRF']],
    ["range", ['FitRatioHPT']],
    ["range", ['ptResolHPT']],
    ["range", ['snp0']],
    #
    ['multiSelect',["isSecondary"]],
    ['multiSelect',["isHighPt"]],
    ['multiSelect',["dir1"]],
    ['multiSelect',["X0NormC"]],
    ['multiSelect',["sigmaRPhiC"]],
    ['multiSelect',["pidCode"]],
#    ['multiSelect',["sigmaRPhiC"]],
    #dummy
]
widgetLayoutDescT0=[  
                    [3,4,5,6,7],
                    [8,9,10,11],
                    [12,13,14,15],
                     [16,17,18,19,20,21],
                    {'sizing_mode':'scale_width',"legend_visible":True} ]
widgetLayoutDesc={
    "Selection": widgetLayoutDescT0,
    "Graphics": [[0,1,2], {'sizing_mode': 'scale_width'}]
}

In [ ]:
histoArray = [
    #
    {"name": "histoqPtLarm", "variables": ["sigmaPtRel0","Larm"], "nbins": [50,20]},
    {"name": "histoqPtSRPhi", "variables": ["sigmaPtRel0","sigmaRPhi"], "nbins": [50,20]},
    #
    {"name": "histoqPt", "variables": ["qPt"], "nbins": 50},
    {"name": "histoFitRatioRF", "variables": ["FitRatioRF"], "nbins": 50},
    {"name": "histoFitRatioHPT", "variables": ["FitRatioHPT"], "nbins": 50},
    {"name": "histoFitRatioLPT", "variables": ["FitRatioLPT"], "nbins": 50},
    #
]

figureArray = [
    # tab 0: pt:qpt resolution :
    [['qPt'], ['sigmaPtRel0'], {"colorZvar":"colorZPt"}],
    [['qPt'], ['sigmaPtRel1_Median'], {"colorZvar":"colorZPt"}],
    [['qPt'], ['ptResolHPT'], {"colorZvar":"colorZPt"}],
    [['qPt'], ['ptResolLPT'], {"colorZvar":"colorZPt"}],
    # tab1: pt:pt resolution
    [['ptMC'], ['sigmaPtRel0'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['sigmaPtRel1_Median'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['ptResolHPT'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['ptResolLPT'], {"colorZvar":"colorZPt"}],
    # tab2 : qpt:pt resolution
    [['ptMC'], ['sigmaqPt0'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['sigmaPtRel1_Median'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['qptResolHPT'], {"colorZvar":"colorZPt"}],
    [['ptMC'], ['qptResolLPT'], {"colorZvar":"colorZPt"}],
    # tab3; sigmaqPt scaling
    [['Larm**(-2.5)'], ['sigmaqPt0/sigmaRPhi'], {"colorZvar":"colorZPt"}],
    [['sigmaRPhi'], ['sigmaqPt0*(Larm**(2.5))'], {"colorZvar":"colorZPt"}],
    [['Larm'], ['(sigmaqPt0/sigmaRPhi)*(Larm**(2.5))'], {"colorZvar":"colorZPt"}],
    [['sigmaRPhi'], ['(sigmaqPt0/sigmaRPhi)*(Larm**(2.5))'], {"colorZvar":"colorZPt"}],
    # tab4; sigmaPtRel scaling  (LArm, dEdxExp, 1/pt, X0)
    [['Larm'], ['(sigmaPtRel0/sqrt(X0Norm))*Larm'], {"colorZvar":"colorZPt"}],
    [['X0Norm'], ['(sigmaPtRel0/sqrt(X0Norm))*Larm'], {"colorZvar":"colorZPt"}],
    [['sqrt(dEdxExp)'], ['(sigmaPtRel0/sqrt(X0Norm))*Larm'], {"colorZvar":"colorZPt"}],
    [['1/ptMC'], ['(sigmaPtRel0/sqrt(X0Norm))*Larm'], {"colorZvar":"colorZPt"}],
    #[['1/ptMC'], ['(sigmaPtRel0/sqrt(X0Norm))*Larm'], {"colorZvar":"colorZPt"}],
    #
    [['qPt'], ['FitRatioRF'], {"colorZvar":"pidCode"}],
    [['qPt'], ['FitRatioHPT'], {"colorZvar":"pidCode"}],
    [['qPt'], ['FitRatioLPT'], {"colorZvar":"pidCode"}],
    #
    [["qPt"],["histoqPt"],{}],
    [["FitRatioRF"],["histoFitRatioRF"],{}],
    [["FitRatioHPT"],["histoFitRatioHPT"],{}],
    [["FitRatioLPT"],["histoFitRatioLPT"],{}],
    ["tableHisto", {"rowwise": False}],
    ['table'],
    {"size": 5}
]

tooltips = [("qPt","@qPt"),("sigmaRPhi","@sigmaRPhi")]
figureLayoutDescT0=[ [0,1,  {'plot_height':200, "commonX":3}],     [2,3,  {'plot_height':200, "commonX":3}],       {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}]
figureLayoutDescT1=[ [4,5,  {'plot_height':200, "commonX":7}],     [6,7,  {'plot_height':200, "commonX":7}],       {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True}]
figureLayoutDescT2=[ [8,9,  {'plot_height':200, "commonX":11}],    [10,11,  {'plot_height':200, "commonX":11}],        {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True} ]
figureLayoutDescT3=[ [12,13,  {'plot_height':200}],    [14,15,  {'plot_height':200}],        {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True} ]
figureLayoutDescT4=[ [16,17,  {'plot_height':200}],    [18,19,  {'plot_height':200}],        {'plot_height':200,'sizing_mode':'scale_width',"legend_visible":True} ]

#
figureLayoutDesc={
    "pt:qpt Resol tab": figureLayoutDescT0,
    "pt:pt Resol  tab": figureLayoutDescT1,
    "qpt:pt Resol tab": figureLayoutDescT2,
    "qpt: scaling Resol (high pt variables)": figureLayoutDescT3,
    "qpt: scaling Resol (low pt scaling)": figureLayoutDescT4,
    #"Histos": figureLayoutDescT2, 
}
output_file("fastSimulDisplayResolution.html")


figC=bokehDrawSA.fromArray(dfResMLDraw, "fMaxLayer>=0&ptMC<50&Larm>5", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=8000,
                           rescaleColorMapper=True,arrayCompression=arrayCompressionRelative8,histogramArray=histoArray,parameterArray=parameterArray)
#figC=bokehDrawSA.fromArray(dfResMLDraw.sample(1000), "fMaxLayer>=0&ptMC<50&Larm>5", figureArray, widgetParams,layout=figureLayoutDesc,tooltips=tooltips,sizing_mode='scale_width',widgetLayout=widgetLayoutDesc,nPointRender=2000,
#                           rescaleColorMapper=True,histogramArray=histoArray,parameterArray=parameterArray)


In [ ]:
dfResMLDraw[1515:1520].head

In [ ]:
dfResMLDraw["pMC"]=dfResMLDraw.eval("sqrt(1+tglMC**2)*ptMC")
dfResMLDraw["Beta"]=dfResMLDraw.eval("sqrt(pMC**2/(pMC**2+fMassMC**2))")
dfResMLDraw["ptResolLPT"]=dfResMLDraw.eval("(0.2*0.5/Beta)*sqrt(Larm/100.)")

In [ ]:
dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*(sigmaRPhi+0.01)*sqrt(720./(Larm+4.))/(0.3*0.5*(Larm/100.)**2.))")

In [ ]:
(dfResMLDraw["FitRatioLPT"][1515:1520]==-np.inf)
np.nan_to_num((dfResMLDraw["FitRatioLPT"][1515:1520]),0)==-np.inf

# Fast prototyping part

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
rfVars=['sigmaPtRel1_Mean','sigmaPtRel1_Median']
#dfResML["sigmaPtRel1_Median"]
#dfResML.plot(x="sigmaPtRel1_Median", y="sigmaPtRel1_Median",figsize=(30, 6), grid=True)
dfResML.plot(x="sigmaPtRel1_Median", y="sigmaPtRel1_Median")


In [ ]:
dfResML["sigmaPtRel1_Median"]

In [ ]:
dfResML.plot?

In [ ]:

dfResMLDraw["ptResolHPT"]=dfResMLDraw.eval("(abs(1/qPt)*sigmaRPhi*sqrt(720.)/(0.5*Larm**(2.5)))")

In [ ]:
dfResMLDraw["X0Norm"].hist()

In [ ]:
dfResMLDraw

# Debug section

In [ ]:
tree.Draw("LArm","Larm>5&&ptMC>10&&(!isSecondary)","prof")

# Resolution scaling test -high pt

In [ ]:
# t1. Resolution scaling does not work at all for secondaries
# t2. Resolution scaling works like expected (L**2.5) down to Larm 100 cm, below it is breaking in case fiitCut used 'ptMC>0.01&&Larm>30&&sigmaPtRel0<0.5&&fMaxLayer>2'
# t3.  Resolution scaling works like expected (L**2.5) over all LArm region 10-250 cm 
#t1.
#tree.Draw("sigmaqPt0*(sigmaRPhi+0.001)*Larm**2.5:Larm>>hisLArm(25,0,250)","Larm>5&&ptMC>10&&(isSecondary)","prof")
#t2.
#tree.Draw("sigmaqPt0*(sigmaRPhi+0.001)*Larm**2.5:Larm>>hisLArm(25,0,250)","Larm>5&&ptMC>10&&(!isSecondary)&&fitCut","prof")
#t3.
tree.Draw("sigmaqPt0*(sigmaRPhi+0.001)*Larm**2.5:Larm>>hisLArm(25,0,250)","Larm>5&&ptMC>10&&(!isSecondary)","prof")

c.Draw("")

# low pt scaling test
* sigmaPtRel0*Larm 

In [ ]:
# t1)  proportianal to 1/Larm
#tree.Draw("sigmaPtRel0/(1/Larm) :Larm","Larm>40&&ptMC<2&&(!isSecondary)","prof")
#t2) proportional to 
tree.Draw("sigmaPtRel0*sqrt(Larm):sqrt(X0Norm)","Larm>20&&ptMC<2&&(!isSecondary)","prof")
#tree.Draw("sigmaPtRel0/(X0Norm*sqrt(dEdxExp)/Larm):X0Norm*sqrt(dEdxExp)","Larm>40&&ptMC<1&&(!isSecondary)","prof")
# t3) proportional to X0Norm*sqrt(dEdxExp)

c.Draw("")

In [ ]:
tree.GetEntries()

In [ ]:
tree.GetAlias("fitCut")

In [ ]:
dfResMLDraw["dEdxExp"].hist()

In [ ]:
tree.SetAlias("meandMSA","part.getMean(2+0,0,1)");
tree.SetAlias("meandMSB","part.getMean(2+0,0,0.2)");
tree.Draw("sigmaqPt0*Larm:sqrt(part.getMean(4+0,0,0.5)*X0Norm):fMassMC","Larm>5&&ptMC<1&&Larm<250&&sigmaPtRel0<1&&(!isSecondary)&&sqrt(part.getMean(4+0,0,0.9))<150&&fMassMC<1","colz",10000)
#tree.Draw("sigmaqPt0*sqrt(Larm)/sqrt(part.getMean(4+0,0,0.8)):sqrt(part.getMean(4+0,0,0.8))","Larm>5&&ptMC<1&&Larm<300&&sigmaPtRel0<1&&(!isSecondary)&&sqrt(part.getMean(4+0,0,0.9))<150&&fMassMC<1","prof",10000)
#tree.Draw("sqrt(part.getMean(3+0,0,0.9))/part.getMean(0+0,0,0.9):sqrt(part.getMean(4+0,0,0.9)):fPdgCodeMC","Larm>4&&ptMC<1&&Larm>50&&sigmaPtRel0<1&&(!isSecondary)&&sqrt(part.getMean(4+0,0,0.6))<150","colz",10000)
#tree.Draw("sigmaqPt0*sqrt(X0Norm*Larm)","Larm>10&&ptMC<1&&Larm<50&&sigmaPtRel0<1&&(!isSecondary)&&sqrt(part.getMean(4+0,0,0.9))<150&&fMassMC<1","colz",10000)
tree.GetHistogram().Fit("pol1")
c.Draw("")

# Secondary debugging:
* track looks to be stack
* status to be checked 
*  in case connection lost -   kTrackUpdate =0x8, 
*  sometime not all points used in refit - sometimes ==0
*  gdb: 


In [ ]:
tree.SetAlias("isOK","part.fStatusMaskIn.fData[1]==31&&part.fLengthIn>100");


tree.Draw("sigmaqPt0:fLengthIn:sigmaRPhi","!isSecondary&&ptMC>4&&fLengthIn>100&&isOK","colz",10000)                      # nonsece pt pt depdence
# tree.Draw("sigmaqPt0:part.fParamIn[1].fX","isSecondary&&ptMC>4&&Larm>150","",10000)
# tree.Draw("part.fParamIn.fX-part.fParamMC.fX","isSecondary&&ptMC>4&&Larm>150","",10000)   #in 30 % of cases mismath in position, in MC data
#tree.Draw("part.fStatusMaskIn.fData[2]","isSecondary&&ptMC>4&&Larm>150","",10000)          # in 20 % of case -update failed
c.Draw()

In [ ]:
tree.GetAlias("sigmaqPt0")

In [ ]:
tree.Draw("sqrt(part.fParamIn[0].fC[14])*(Larm**2.5)/sigmaRPhi:part.fParamIn[0].fP[2]","ptMC>5&&(isSecondary)&&Larm>50&&sigmaRPhi>0.02","prof")
c.Draw()

In [ ]:
tree.Draw("(fLengthIn-Larm)>3","!isSecondary")
c.Draw()

In [ ]:
tree.Show(0)

In [ ]:
tree.Draw("getStat(2):getStat(1)","!isSecondary")
c.Draw()